In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import scipy.stats as stats

In [3]:
raw_data = pd.read_csv('../data/accepted_2007_to_2018Q4.csv.gz', compression='gzip')

C:\Users\juanf\AppData\Local\Temp\ipykernel_1844\803838548.py:1: DtypeWarning: Columns (0,19,49,59,118,129,130,131,134,135,136,139,145,146,147) have mixed types. Specify dtype option on import or set low_memory=False.
  raw_data = pd.read_csv('data/accepted_2007_to_2018Q4.csv.gz', compression='gzip')


In [4]:
# Drop columns from 'title' to the end
start_index= raw_data.columns.get_loc('title')
acc_set = raw_data.iloc[:, :start_index]

acc_set = acc_set.drop(['member_id','funded_amnt_inv','installment','sub_grade','funded_amnt','id','url','desc','pymnt_plan','emp_title','issue_d','verification_status'],axis=1)
acc_set = acc_set.drop(['home_ownership', 'emp_length'], axis=1)

In [5]:
acc_set.head()

,loan_amnt,term,int_rate,grade,annual_inc,loan_status,purpose
0,3600.0,36 months,13.99,C,55000.0,Fully Paid,debt_consolidation
1,24700.0,36 months,11.99,C,65000.0,Fully Paid,small_business
2,20000.0,60 months,10.78,B,63000.0,Fully Paid,home_improvement
3,35000.0,60 months,14.85,C,110000.0,Current,debt_consolidation
4,10400.0,60 months,22.45,F,104433.0,Fully Paid,major_purchase


DATASET OVERVIEW

In [6]:
acc_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2260701 entries, 0 to 2260700
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   loan_amnt    float64
 1   term         object 
 2   int_rate     float64
 3   grade        object 
 4   annual_inc   float64
 5   loan_status  object 
 6   purpose      object 
dtypes: float64(3), object(4)
memory usage: 120.7+ MB


ADDING MISSING VALUES (TRAIN DATASET)

In [7]:
# Check for missing values
acc_set.isnull().sum()

loan_amnt      33
term           33
int_rate       33
grade          33
annual_inc     37
loan_status    33
purpose        33
dtype: int64

In [8]:
missing_percentage = (acc_set.isnull().sum() / len(acc_set)) * 100
# Create DF to display the results
missing_data = pd.DataFrame({
    'Variable': missing_percentage.index,
    'Missing Percentage': missing_percentage.values
})
# Sort the DF by missing percentage
missing_data = missing_data.sort_values(by='Missing Percentage', ascending=False)
print(missing_data)

      Variable  Missing Percentage
4   annual_inc            0.001637
0    loan_amnt            0.001460
1         term            0.001460
2     int_rate            0.001460
3        grade            0.001460
5  loan_status            0.001460
6      purpose            0.001460


In [9]:
# Drop missing values since the data was large enough
acc_set = acc_set.dropna()

In [10]:
acc_set['loan_amnt'].describe()

count    2.260664e+06
mean     1.504695e+04
std      9.190246e+03
min      5.000000e+02
25%      8.000000e+03
50%      1.290000e+04
75%      2.000000e+04
max      4.000000e+04
Name: loan_amnt, dtype: float64

In [11]:
acc_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2260664 entries, 0 to 2260698
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   loan_amnt    float64
 1   term         object 
 2   int_rate     float64
 3   grade        object 
 4   annual_inc   float64
 5   loan_status  object 
 6   purpose      object 
dtypes: float64(3), object(4)
memory usage: 138.0+ MB


In [12]:
#REMOVE OUTLIERS
Q1 = acc_set['loan_amnt'].quantile(0.25)
Q3 = acc_set['loan_amnt'].quantile(0.75)
IQR = Q3 - Q1
acc_set = acc_set[~((acc_set['loan_amnt'] < (Q1 - 1.5 * IQR)) | (acc_set['loan_amnt'] > (Q3 + 1.5 * IQR)))]


In [13]:
#REMOVE OUTLIERS
Q1 = acc_set['int_rate'].quantile(0.25)
Q3 = acc_set['int_rate'].quantile(0.75)
IQR = Q3 - Q1
acc_set = acc_set[~((acc_set['int_rate'] < (Q1 - 1.5 * IQR)) | (acc_set['int_rate'] > (Q3 + 1.5 * IQR)))]

In [14]:
acc_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 2183697 entries, 0 to 2260698
Data columns (total 7 columns):
 #   Column       Dtype  
---  ------       -----  
 0   loan_amnt    float64
 1   term         object 
 2   int_rate     float64
 3   grade        object 
 4   annual_inc   float64
 5   loan_status  object 
 6   purpose      object 
dtypes: float64(3), object(4)
memory usage: 133.3+ MB


In [15]:
#REMOVE OUTLIERS
Q1 = acc_set['annual_inc'].quantile(0.25)
Q3 = acc_set['annual_inc'].quantile(0.75)
IQR = Q3 - Q1
acc_set = acc_set[~((acc_set['annual_inc'] < (Q1 - 1.5 * IQR)) | (acc_set['annual_inc'] > (Q3 + 1.5 * IQR)))]

In [16]:
print(acc_set["loan_amnt"].skew())
print(acc_set["int_rate"].skew())
print(acc_set["annual_inc"].skew())

# int_rate is fairly symmetrical (between -0.5 and 0.5)
# loan_amnt & annual_inc are moderately skewed (between 0.5 and 1)

0.7416443994369606
0.46701548924966996
0.7203586672756999


In [17]:
category_column = 'loan_status'

# Count occurrences of each category in the specified column
category_counts = acc_set['loan_status'].value_counts()

# Display the counts
print(category_counts)

loan_status
Fully Paid                                             1011325
Current                                                 790899
Charged Off                                             248396
Late (31-120 days)                                       18786
In Grace Period                                           7275
Late (16-30 days)                                         3757
Does not meet the credit policy. Status:Fully Paid        1881
Does not meet the credit policy. Status:Charged Off        715
Default                                                     34
Name: count, dtype: int64


In [18]:
#Filtered to only Current and Fully Paid loan statuses
filtered_acc_set = acc_set[acc_set['loan_status'].isin(['Fully Paid', 'Charged Off'])]

filtered_acc_set.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1259721 entries, 0 to 2260697
Data columns (total 7 columns):
 #   Column       Non-Null Count    Dtype  
---  ------       --------------    -----  
 0   loan_amnt    1259721 non-null  float64
 1   term         1259721 non-null  object 
 2   int_rate     1259721 non-null  float64
 3   grade        1259721 non-null  object 
 4   annual_inc   1259721 non-null  float64
 5   loan_status  1259721 non-null  object 
 6   purpose      1259721 non-null  object 
dtypes: float64(3), object(4)
memory usage: 76.9+ MB


In [19]:
filtered_acc_set_paid = acc_set[acc_set['loan_status'].isin(['Fully Paid'])]
filtered_acc_set_charged = acc_set[acc_set['loan_status'].isin(['Charged Off'])]


In [20]:
Q1 = filtered_acc_set_paid['annual_inc'].quantile(0.25)
Q3 = filtered_acc_set_paid['annual_inc'].quantile(0.75)
IQR = Q3 - Q1
filtered_acc_set_paid = filtered_acc_set_paid[~((filtered_acc_set_paid['annual_inc'] < (Q1 - 1.5 * IQR)) | (filtered_acc_set_paid['annual_inc'] > (Q3 + 1.5 * IQR)))]

In [21]:
Q1 = filtered_acc_set_paid['loan_amnt'].quantile(0.25)
Q3 = filtered_acc_set_paid['loan_amnt'].quantile(0.75)
IQR = Q3 - Q1
filtered_acc_set_paid = filtered_acc_set_paid[~((filtered_acc_set_paid['loan_amnt'] < (Q1 - 1.5 * IQR)) | (filtered_acc_set_paid['loan_amnt'] > (Q3 + 1.5 * IQR)))]

In [22]:
Q1 = filtered_acc_set_paid['int_rate'].quantile(0.25)
Q3 = filtered_acc_set_paid['int_rate'].quantile(0.75)
IQR = Q3 - Q1
filtered_acc_set_paid = filtered_acc_set_paid[~((filtered_acc_set_paid['int_rate'] < (Q1 - 1.5 * IQR)) | (filtered_acc_set_paid['int_rate'] > (Q3 + 1.5 * IQR)))]

In [23]:
print(filtered_acc_set_paid['int_rate'].skew())
print(filtered_acc_set_paid['loan_amnt'].skew())
print(filtered_acc_set_paid['annual_inc'].skew())

0.379792378282666
0.6710291454975075
0.6840959402069496


In [24]:
Q1 = filtered_acc_set_charged['annual_inc'].quantile(0.25)
Q3 = filtered_acc_set_charged['annual_inc'].quantile(0.75)
IQR = Q3 - Q1
filtered_acc_set_charged = filtered_acc_set_charged[~((filtered_acc_set_charged['annual_inc'] < (Q1 - 1.5 * IQR)) | (filtered_acc_set_charged['annual_inc'] > (Q3 + 1.5 * IQR)))]

In [25]:
Q1 = filtered_acc_set_charged['loan_amnt'].quantile(0.25)
Q3 = filtered_acc_set_charged['loan_amnt'].quantile(0.75)
IQR = Q3 - Q1
filtered_acc_set_charged = filtered_acc_set_charged[~((filtered_acc_set_charged['loan_amnt'] < (Q1 - 1.5 * IQR)) | (filtered_acc_set_charged['loan_amnt'] > (Q3 + 1.5 * IQR)))]

In [26]:
Q1 = filtered_acc_set_charged['int_rate'].quantile(0.25)
Q3 = filtered_acc_set_charged['int_rate'].quantile(0.75)
IQR = Q3 - Q1
filtered_acc_set_charged = filtered_acc_set_charged[~((filtered_acc_set_charged['int_rate'] < (Q1 - 1.5 * IQR)) | (filtered_acc_set_charged['int_rate'] > (Q3 + 1.5 * IQR)))]

In [27]:
print(filtered_acc_set_charged['annual_inc'].skew())
print(filtered_acc_set_charged['loan_amnt'].skew())
print(filtered_acc_set_charged['int_rate'].skew())

0.6021600317818238
0.6696931802098749
0.21998913458754438


In [28]:
filtered_acc_set = pd.concat([filtered_acc_set_paid, filtered_acc_set_charged])

In [29]:
result = stats.f_oneway(filtered_acc_set_paid['loan_amnt'], filtered_acc_set_charged['loan_amnt'])

print(f'F-statistic: {result.statistic:.6f}')
print(f'P-value: {result.pvalue:.15f}')

# p-value is less than 0.05, so we reject the null hypothesis and conclude that the means of loan amount 
# are significantly different between the people who charged off and the people who fully paid

F-statistic: 12686.245187
P-value: 0.000000000000000


In [30]:
stats.f_oneway(filtered_acc_set_paid['int_rate'] , filtered_acc_set_charged['int_rate'])
# p-value is less than 0.05, so we reject the null hypothesis and conclude that the means of interest rate
# are significantly different between the people who charged off and the people who fully paid

F_onewayResult(statistic=101014.19096358858, pvalue=0.0)

# Machine Learning

In [31]:
model_data = filtered_acc_set.copy()
model_data = model_data.reset_index()

num_var=["loan_amnt","int_rate","annual_inc"]
cat_var=["term","grade","purpose"]

In [32]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

encoder = OneHotEncoder()

one_hot_encoded = encoder.fit_transform(model_data[cat_var])

column_names = encoder.get_feature_names_out(cat_var)

one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=column_names)
data_encoded = pd.concat([model_data, one_hot_encoded_df], axis=1)
data_encoded.drop(columns=cat_var, inplace=True,errors='ignore')
data_encoded.drop(columns=['index'], inplace=True,errors='ignore')

model_data=data_encoded

In [33]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(model_data.drop('loan_status', axis=1), model_data['loan_status'], test_size=0.2, random_state=0)

In [34]:
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import roc_auc_score
from sklearn.metrics import roc_curve
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc

# Classifier

## Random Forest

In [35]:
from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(n_estimators=10, min_samples_leaf=8, random_state=0)

# Train the model
rf.fit(X_train, y_train)

# Evaluate the model on the train and testing set
y_train_pred = rf.predict(X_train)
y_test_pred = rf.predict(X_test)

In [36]:
acc1 = accuracy_score(y_test, y_test_pred)
acc1

0.8113998845644383

In [171]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

new_input = pd.DataFrame({
    'loan_amnt': [50000],
    'term': [' 36 months'],
    'int_rate': [10],
    'grade': ['A'],
    'annual_inc': [10000],
    'purpose': ['debt_consolidation']
})

new_input_cat = new_input[cat_var]
new_input_num = new_input[num_var]

new_input_encoded = encoder.transform(new_input_cat)
new_input_encoded = pd.DataFrame(new_input_encoded.toarray(), columns=column_names)
new_input_encoded = pd.concat([new_input_num.reset_index(drop=True), new_input_encoded.reset_index(drop=True)], axis=1)

predictions = rf.predict(new_input_encoded)
predictions

array(['Charged Off'], dtype=object)

# KNN

In [41]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors=3)

knn_model = knn.fit(X_train, y_train)
knn_pred = knn_model.predict(X_test)

print('Accuracy Score:', accuracy_score(y_test, knn_pred))

Accuracy Score: 0.7594912530571738


In [123]:
filtered_acc_set.head()

,loan_amnt,term,int_rate,grade,annual_inc,loan_status,purpose
0,3600.0,36 months,13.99,C,55000.0,Fully Paid,debt_consolidation
1,24700.0,36 months,11.99,C,65000.0,Fully Paid,small_business
2,20000.0,60 months,10.78,B,63000.0,Fully Paid,home_improvement
4,10400.0,60 months,22.45,F,104433.0,Fully Paid,major_purchase
5,11950.0,36 months,13.44,C,34000.0,Fully Paid,debt_consolidation


In [154]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

new_input = pd.DataFrame({
    'loan_amnt': [30000],
    'term': [' 60 months'],
    'int_rate': [17],
    'grade': ['F'],
    'annual_inc': [25000],
    'purpose': ['major_purchase']
})

new_input_cat = new_input[cat_var]
new_input_num = new_input[num_var]

new_input_encoded = encoder.transform(new_input_cat)
new_input_encoded = pd.DataFrame(new_input_encoded.toarray(), columns=column_names)
new_input_encoded = pd.concat([new_input_num.reset_index(drop=True), new_input_encoded.reset_index(drop=True)], axis=1)

predictions = knn_model.predict(new_input_encoded)
predictions

array(['Charged Off'], dtype=object)

# Predicting Maximum Loan Amount

## Filtering Data

In [338]:
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import ColumnTransformer

paid_data = filtered_acc_set_paid
paid_data.dropna(inplace=True)
paid_data.head()

encoder = OneHotEncoder()

# Fit and transform the data
one_hot_encoded = encoder.fit_transform(paid_data[cat_var])

column_names = encoder.get_feature_names_out(cat_var)

# concatenate the DataFrame with the one hot encoded values
one_hot_encoded_df = pd.DataFrame(one_hot_encoded.toarray(), columns=column_names)
data_encoded = pd.concat([paid_data, one_hot_encoded_df], axis=1)
data_encoded.drop(columns=cat_var, inplace=True,errors='ignore')
data_encoded.drop(columns=['index','loan_status'], inplace=True,errors='ignore')
data_encoded.dropna(inplace=True)

paid_data=data_encoded

In [339]:
X_train_regression, X_test_regression, y_train_regression, y_test_regression = train_test_split(paid_data.drop('loan_amnt', axis=1), paid_data['loan_amnt'], test_size=0.2, random_state=0)

## Random Forest Regressor

In [340]:
from sklearn.ensemble import RandomForestRegressor

rf_reg = RandomForestRegressor(n_estimators=10, min_samples_leaf=16, random_state=0)

rf_reg.fit(X_train_regression, y_train_regression)
rf_pred_train = rf_reg.predict(X_train_regression)
rf_pred_test = rf_reg.predict(X_test_regression)

from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
print("Explained Variance (R^2) \t:", r2_score(y_test_regression, rf_pred_test))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test_regression, rf_pred_test))
print("Explained Variance (R^2) \t:", r2_score(y_train_regression, rf_pred_train))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train_regression, rf_pred_train))

Explained Variance (R^2) 	: 0.19480497299815203
Mean Squared Error (MSE) 	: 43486608.96240037
Explained Variance (R^2) 	: 0.2864862589860334
Mean Squared Error (MSE) 	: 38780703.79696835


In [350]:
new_input = pd.DataFrame({
    'term': [' 36 months'],
    'int_rate': [15],
    'grade': ['A'],
    'emp_length': ['10+ years'],
    'home_ownership': ['NONE'],
    'annual_inc': [1000],
    'verification_status': ['Verified'],
    'issue_d': ['2018-12-01'],
    'purpose': ['small_business']
})

new_input_cat = new_input[cat_var]
new_input_num = new_input[["int_rate","annual_inc"]]

new_input_encoded = encoder.transform(new_input_cat)
new_input_encoded = pd.DataFrame(new_input_encoded.toarray(), columns=column_names)
new_input_encoded = pd.concat([new_input_num.reset_index(drop=True), new_input_encoded.reset_index(drop=True)], axis=1)

predictions = rf_reg.predict(new_input_encoded)
predictions[0]

15568.429093597737

## XGBoost

In [250]:
import xgboost as xgb

dtrain = xgb.DMatrix(X_train_regression, label=y_train_regression)
dtest = xgb.DMatrix(X_test_regression, label=y_test_regression)

param = {'max_depth': 3, 'learning_rate': 0.1, 'objective': 'reg:squarederror'}
num_round = 250

xgb_model = xgb.train(param, dtrain, num_round)

y_train_pred = xgb_model.predict(dtrain)
y_test_pred = xgb_model.predict(dtest)

print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", r2_score(y_train_regression, y_train_pred))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train_regression, y_train_pred))
print()

print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", r2_score(y_test_regression, y_test_pred))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test_regression, y_test_pred))

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.23786215644022946
Mean Squared Error (MSE) 	: 18.62832368084702

Goodness of Fit of Model 	Test Dataset
Explained Variance (R^2) 	: 0.23866974708597077
Mean Squared Error (MSE) 	: 18.66739061973875


## Elastic Learn

In [253]:
from sklearn.datasets import make_regression
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split


reg = GradientBoostingRegressor(random_state=0)
reg.fit(X_train_regression, y_train_regression)
reg.predict(X_test_regression[1:2])

print("Goodness of Fit of Model \tTrain Dataset")
print("Explained Variance (R^2) \t:", r2_score(y_train_regression, y_train_pred))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_train_regression, y_train_pred))
print()

print("Goodness of Fit of Model \tTest Dataset")
print("Explained Variance (R^2) \t:", r2_score(y_test_regression, y_test_pred))
print("Mean Squared Error (MSE) \t:", mean_squared_error(y_test_regression, y_test_pred))

Goodness of Fit of Model 	Train Dataset
Explained Variance (R^2) 	: 0.23786215644022946
Mean Squared Error (MSE) 	: 18.62832368084702

Goodness of Fit of Model 	Test Dataset
Explained Variance (R^2) 	: 0.23866974708597077
Mean Squared Error (MSE) 	: 18.66739061973875
